In [1]:
import os
import pandas as pd
import numpy as np
import time
import cv2
import tensorflow as tf
import keras.backend as K
import keras

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [2]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7111709695699251610
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3381723136
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12456200975974913950
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
]


# Data Preprocessing

In [3]:
def generate_training_data(output_type):
    data_path="dataset/"
    data=pd.read_csv(data_path+"label.csv")
    file_names=list(data['file_name'])
    labels=list(data['label'])
    
    if output_type == 'ohe':
        data['tumor_label'] = data['label'].apply(lambda x: "no_tumor" if x == "no_tumor" else "tumor")
        data = data.drop(columns = ['label'])
        labels=data['tumor_label'].values.reshape(-1,1)
        ohe = OneHotEncoder(handle_unknown = "ignore", sparse=False)
        labels = ohe.fit_transform(labels)
    
    if output_type == 'binary':
        data['tumor_label'] = data['label'].apply(lambda x: 0 if x == "no_tumor" else 1)
        data = data.drop(columns = ['label'])
        labels=data['tumor_label'].values.reshape(-1,1)
        
    dataset_tumor=[]

    for file_index in range(len(file_names)):
        file=cv2.imread(data_path+"/image/"+file_names[file_index])
        file_resize=cv2.resize(file,(128,128))/255.
        dataset_tumor.append(file_resize)
        
    tumor_data = np.array(dataset_tumor)
    
    return tumor_data, labels

In [4]:
tumor_data, labels = generate_training_data('binary')
x_train,x_test,y_train,y_test=train_test_split(tumor_data,labels,test_size=0.2,random_state=25)

# Training and Validation

In [5]:
x_train = x_train.reshape(2400,128*128*3)
x_test = x_test.reshape(600,128*128*3)
y_train = y_train.ravel()

In [6]:
#scaler = MinMaxScaler()
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [11]:
def logRegrPredict(x_train, y_train,xtest ):

    logreg = LogisticRegression(solver='saga')
    logreg.fit(x_train, y_train)
    y_pred= logreg.predict(xtest)

    return y_pred

y_pred = logRegrPredict(x_train, y_train,x_test)
print('Accuracy on test set: '+str(accuracy_score(y_test,y_pred)))
print(classification_report(y_test,y_pred))#text report showing the main classification metrics

Accuracy on test set: 0.8566666666666667
              precision    recall  f1-score   support

           0       0.47      0.88      0.61        77
           1       0.98      0.85      0.91       523

    accuracy                           0.86       600
   macro avg       0.72      0.87      0.76       600
weighted avg       0.91      0.86      0.87       600



C:\Software\Anaconda\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
